In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import math

In [5]:
discussions_df = pd.read_csv('Discussions.csv', header=0, encoding='utf-8')
ratings_df = pd.read_csv('Ratings.csv', header=0, encoding='utf-8')
display(discussions_df)
display(ratings_df)

,DiscussionID,AlbumName,ArtistName,Date,AvgRating,Stdev,Attendance,RotationGenre,OtherGenre,Subgenres,ReleaseYear,FavoriteTrack,Popularity,Tracks,SpotifyID,Image
0,1.0,IGOR,"Tyler, the Creator",2022-02-19,0.00,0.00,0.0,NaN,False,"Neo-Soul, Psychedelic Soul",2019.0,NaN,85.0,12.0,5zi7WsKlIiUXv09tbGLKsE,https://i.scdn.co/image/ab67616d0000b273700588...
1,2.0,The New Abnormal,The Strokes,2022-02-26,0.00,0.00,0.0,NaN,False,Indie Rock,2020.0,NaN,76.0,9.0,2xkZV2Hl1Omi8rk2D7t5lN,https://i.scdn.co/image/ab67616d0000b273bfa99a...
2,3.0,Blonde,Frank Ocean,2022-03-06,0.00,0.00,0.0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",2016.0,NaN,86.0,17.0,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...
3,4.0,So Tonight that I Might See,Mazzy Star,2022-03-27,0.00,0.00,0.0,NaN,False,"Dream Pop, Neo-Psychedelia",1993.0,NaN,73.0,10.0,5K18gTgac0q6Jma5HkV1vA,https://i.scdn.co/image/ab67616d0000b27389a392...
4,5.0,A New World Record,Electric Light Orchestra,2022-04-03,0.00,0.00,0.0,NaN,False,"Symphonic Rock, Pop Rock, Progressive Pop",1976.0,NaN,67.0,9.0,7a35UzxXYuKQGMGImyB0Un,https://i.scdn.co/image/ab67616d0000b273ee5b10...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,67.0,Titanic Rising,Weyes Blood,2024-02-05,8.20,1.50,55.0,Pop,False,"Baroque Pop, Art Pop, Progressive Pop, Singer-...",2019.0,Andromeda,68.0,10.0,0Cuqhgy8vm96JEkBY3polk,https://i.scdn.co/image/ab67616d00001e020c64e7...
67,68.0,Because the Internet,Childish Gambino,2024-02-12,6.88,1.93,42.0,Hip-Hop,False,"Abstract Hip Hop, West Coast Hip Hop, Contempo...",2013.0,3005,81.0,19.0,62yjWIhnATHxPqGbgC9Lqr,https://i.scdn.co/image/ab67616d00001e0226d64b...
68,69.0,The Four Seasons,Antonio Vivaldi,2024-02-19,8.08,1.25,38.0,Classical,True,"Baroque Music, Concerto",1985.0,Winter: I. Allegro non molto,63.0,12.0,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...
69,70.0,House Of Balloons (Original),The Weeknd,2024-02-26,8.04,1.08,30.0,Mainstream,True,"Alternative R&B, Downtempo",2011.0,House Of Balloons / Glass Table Girls,83.0,9.0,7zCODUHkfuRxsUjtuzNqbd,https://i.scdn.co/image/ab67616d00001e02274b40...


,MemberID,DiscussionID,Rating,FavoriteTrack1,FavoriteTrack2,FavoriteTrack3
0,88,1,7,NaN,NaN,NaN
1,59,2,8,NaN,NaN,NaN
2,88,3,9,NaN,NaN,NaN
3,294,3,10,NaN,NaN,NaN
4,201,3,7,NaN,NaN,NaN
...,...,...,...,...,...,...
1707,313,69,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."
1708,165,69,7,NaN,NaN,NaN
1709,131,69,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."
1710,286,69,7,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."


# Idea 1: Subgenres

In the AOTW database, we already have a column "Genre" which we use to designate different rotation cycles (rock, pop, hip hop, other/special, wildcard) so that we see a wide variety of genres in the club. We store a specific album's genres/styles as "subgenres" in the AOTW database, with genres provided by RateYourMusic.

In [4]:
subgenres_series = discussions_df['Subgenres'].str.split(', ').explode()

# Get unique subgenres and sort them alphabetically
subgenres_series = subgenres_series.dropna()
unique_subgenres = sorted(subgenres_series.unique())

# Print unique subgenres
print("Unique Subgenres (Alphabetically Sorted):")
#for subgenre in unique_subgenres:
#    print(subgenre)

Unique Subgenres (Alphabetically Sorted):


In [7]:
merged_df = pd.merge(discussions_df, ratings_df, on='DiscussionID')

# Split the Subgenres column and explode it into multiple rows
merged_df['RotationGenre'] = merged_df['RotationGenre'].str.split(', ')
merged_df = merged_df.explode('RotationGenre')
display(merged_df)

# Group by MemberID and Subgenre, then calculate average rating
average_ratings_by_subgenre = merged_df.groupby(['MemberID', 'RotationGenre'])['Rating'].mean().reset_index()

# Display the DataFrame with average ratings by subgenre
#for idx, row in average_ratings_by_subgenre.iterrows():
#    print(f"{row['MemberID']}\t{row['RotationGenre']}\t{row['Rating']}")

,DiscussionID,AlbumName,ArtistName,Date,AvgRating,Stdev,Attendance,RotationGenre,OtherGenre,Subgenres,...,Popularity,Tracks,SpotifyID,Image,Decade,MemberID,Rating,FavoriteTrack1,FavoriteTrack2,FavoriteTrack3
0,1.0,IGOR,"Tyler, the Creator",2022-02-19,0.00,0.00,0.0,NaN,False,"Neo-Soul, Psychedelic Soul",...,85.0,12.0,5zi7WsKlIiUXv09tbGLKsE,https://i.scdn.co/image/ab67616d0000b273700588...,2010s,88,7,NaN,NaN,NaN
1,2.0,The New Abnormal,The Strokes,2022-02-26,0.00,0.00,0.0,NaN,False,Indie Rock,...,76.0,9.0,2xkZV2Hl1Omi8rk2D7t5lN,https://i.scdn.co/image/ab67616d0000b273bfa99a...,2020s,59,8,NaN,NaN,NaN
2,3.0,Blonde,Frank Ocean,2022-03-06,0.00,0.00,0.0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",...,86.0,17.0,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...,2010s,88,9,NaN,NaN,NaN
3,3.0,Blonde,Frank Ocean,2022-03-06,0.00,0.00,0.0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",...,86.0,17.0,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...,2010s,294,10,NaN,NaN,NaN
4,3.0,Blonde,Frank Ocean,2022-03-06,0.00,0.00,0.0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",...,86.0,17.0,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...,2010s,201,7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,69.0,The Four Seasons,Antonio Vivaldi,2024-02-19,8.08,1.25,38.0,Classical,True,"Baroque Music, Concerto",...,63.0,12.0,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,1980s,313,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."
1708,69.0,The Four Seasons,Antonio Vivaldi,2024-02-19,8.08,1.25,38.0,Classical,True,"Baroque Music, Concerto",...,63.0,12.0,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,1980s,165,7,NaN,NaN,NaN
1709,69.0,The Four Seasons,Antonio Vivaldi,2024-02-19,8.08,1.25,38.0,Classical,True,"Baroque Music, Concerto",...,63.0,12.0,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,1980s,131,8,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."
1710,69.0,The Four Seasons,Antonio Vivaldi,2024-02-19,8.08,1.25,38.0,Classical,True,"Baroque Music, Concerto",...,63.0,12.0,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,1980s,286,7,"Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ...","Vivaldi: The Four Seasons, Violin Concerto in ..."


1	00s	8.0
1	20s-50s	9.0
1	60s	10.0
1	60s-80s	8.0
1	Alt Rock	10.0
1	Country	7.0
1	Dream Pop/Shoegaze	9.0
1	Electric Blues	7.0
1	Experimental	7.0
1	Folk	8.0
1	Hip Hop	8.857142857142858
1	Hip-Hop	7.0
1	Industrial	8.0
1	International/Non-English	8.0
1	Jazz	7.0
1	LGBTQ+	9.0
1	Pop	8.625
1	R&B	10.0
1	Rock	8.777777777777779
1	Small Artist	6.0
1	Wild Card	8.2
1	Wildcard	9.5
2	Rock	8.0
3	Rock	4.0
4	Rock	8.0
5	R&B	10.0
5	Rock	6.0
5	Wild Card	9.0
6	00s	4.0
6	20s-50s	7.0
6	60s	7.0
6	60s-80s	8.0
6	Alt Rock	8.0
6	Country	7.0
6	Electric Blues	7.0
6	Experimental	5.0
6	Folk	7.0
6	Hip Hop	7.333333333333333
6	Hip-Hop	5.0
6	International/Non-English	7.0
6	Jazz	6.0
6	LGBTQ+	9.0
6	Pop	7.25
6	Rock	8.222222222222221
6	Small Artist	8.0
6	Wild Card	8.0
6	Wildcard	7.5
7	Experimental	10.0
7	Hip Hop	9.0
7	Rock	8.5
7	Wild Card	8.0
8	00s	7.0
8	Classical	9.0
8	Hip Hop	10.0
8	Hip-Hop	8.0
8	International/Non-English	7.0
8	Jazz	7.0
8	Pop	6.8
8	Rock	6.75
8	Small Artist	8.0
8	Wildcard	10.0
9	Hip Hop	10.0
9	Pop	9.0
9	Rock	6

In [9]:
X = ratings_df.drop(columns=['Rating'])
y = ratings_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['Rating'] = y_train

merged_df = pd.merge(discussions_df, X_train, on='DiscussionID')
merged_df['RotationGenre'] = merged_df['RotationGenre'].str.split(', ')
merged_df = merged_df.explode('RotationGenre')
average_ratings_by_subgenre = merged_df.groupby(['MemberID', 'RotationGenre'])['Rating'].mean().reset_index()

y_pred = []
for idx, row in X_test.iterrows():
    subgenres = discussions_df[discussions_df['DiscussionID'] == row['DiscussionID']]['RotationGenre'].iloc[0]
    smaller_df = merged_df[merged_df['MemberID'] == row['MemberID']]
    avg_rating = 0
    count = 0
    for sg in subgenres:
        discussions_with_sg = smaller_df[smaller_df['RotationGenre'].apply(lambda x: sg in x)]
        if not discussions_with_sg.empty:
            avg_rating += discussions_with_sg['Rating'].mean()
            count += 1
    if count > 0:
        avg_rating /= count
    else:
        avg_rating = X_train['Rating'].mean()  # Use average rating if no discussions attended by the user have the subgenre
    y_pred.append(int(np.round(avg_rating, 0)))

y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)

TypeError: argument of type 'float' is not iterable

# Idea 2: Decades

If you think about it, many people will gravitate towards a specific 'era' of music (e.g. I tend to gravitate more towards music from the 2010s and 2020s). Let's see what we can get!

In [6]:
discussions_df['Decade'] = discussions_df['ReleaseYear'].astype(str).str[:3] + '0s'

# Split data into train and test sets
X = ratings_df.drop(columns=['Rating'])
y = ratings_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['Rating'] = y_train

# Merge discussions_df with train data
merged_df = pd.merge(discussions_df, X_train, on='DiscussionID')

y_pred = []
for idx, row in X_test.iterrows():
    # Get the decade of the current album
    album_decade = discussions_df.loc[discussions_df['DiscussionID'] == row['DiscussionID'], 'Decade'].iloc[0]
    
    # Filter merged_df to only include discussions attended by the user and in the same decade
    smaller_df = merged_df[(merged_df['MemberID'] == row['MemberID']) & (merged_df['Decade'] == album_decade)]
    
    # Calculate the average rating for discussions in the same decade
    avg_rating = smaller_df['Rating'].mean() if not smaller_df.empty else X_train['Rating'].mean()
    y_pred.append(int(np.round(avg_rating, 0)))

# Calculate RMSE
y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)

RMSE: 1.6833384444561559


# Your Turn: Implement your own content-based filtering!

You can use any of the data already present in the database, or you can use external resources (Musicbrainz, RateYourMusic, Musicboard, Spotify, etc.)

In [ ]:

# Split data into train and test sets
X = ratings_df.drop(columns=['Rating'])
y = ratings_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['Rating'] = y_train

# Merge discussions_df with train data
merged_df = pd.merge(discussions_df, X_train, on='DiscussionID')

y_pred = []
for idx, row in X_test.iterrows():
    # Get the decade of the current album
    album_decade = discussions_df.loc[discussions_df['DiscussionID'] == row['DiscussionID'], 'Decade'].iloc[0]
    
    # Filter merged_df to only include discussions attended by the user and in the same decade
    smaller_df = merged_df[(merged_df['MemberID'] == row['MemberID']) & (merged_df['Decade'] == album_decade)]
    
    # Calculate the average rating for discussions in the same decade
    avg_rating = smaller_df['Rating'].mean() if not smaller_df.empty else X_train['Rating'].mean()
    y_pred.append(int(np.round(avg_rating, 0)))

# Calculate RMSE
y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)

# Split data into train and test sets
X = ratings_df.drop(columns=['Rating'])
y = ratings_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['Rating'] = y_train

# Merge discussions_df with train data
merged_df = pd.merge(discussions_df, X_train, on='DiscussionID')

y_pred = []
for idx, row in X_test.iterrows():
    # Get the decade of the current album
    album_decade = discussions_df.loc[discussions_df['DiscussionID'] == row['DiscussionID'], 'Decade'].iloc[0]
    
    # Filter merged_df to only include discussions attended by the user and in the same decade
    smaller_df = merged_df[(merged_df['MemberID'] == row['MemberID']) & (merged_df['Decade'] == album_decade)]
    
    # Calculate the average rating for discussions in the same decade
    avg_rating = smaller_df['Rating'].mean() if not smaller_df.empty else X_train['Rating'].mean()
    y_pred.append(int(np.round(avg_rating, 0)))

# Calculate RMSE
y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)